In [ ]:
# pip install chembl_webresource_client

## Import packages
The chembl_webresource package should contain the tools to obtain the chEMBL IDs and mechanisms.

In [1]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np

#### Load the list of drugs

In [2]:
druglist_all = pd.read_csv("output/drugslist.csv")

In [3]:
list_1 = druglist_all["."].tolist()

#### chEMBL IDs retrival
**1st round**, the chembl using the 'names' as 'upper case' they appear in the PRISM database, lookig all synonyms and oprefered names.

In [4]:
n_chemb1 = {}

In [6]:
for i in list_1:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i.upper()).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i.upper()).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i.upper()).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb1.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb1.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb1.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb1.update({i : ":("})

In [7]:
def my_filtering_function(pair):
    unwanted_value = ':('
    key, value = pair
    if value != unwanted_value:
        return False  # filter pair out of the dictionary
    else:
        return True  # keep pair in the filtered dictionary

In [8]:
filtered_chembl1 = dict(filter(my_filtering_function, n_chemb1.items()))

In [9]:
filtered_list1 = list(filtered_chembl1.keys())

In [10]:
len(filtered_list1)

412

**2nd round:** Same but with the original names without transfoming to upper

In [11]:
n_chemb2 = {}

In [13]:
for i in filtered_list1:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb2.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb2.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb2.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb2.update({i : ":("})

In [14]:
filtered_chembl2 = dict(filter(my_filtering_function, n_chemb2.items()))

In [15]:
filtered_list2 = list(filtered_chembl2.keys())

In [16]:
len(filtered_list2)

408

**Round 3**: change presence of  '-'  character

In [17]:
n_chemb3 = {}

In [18]:
for i in filtered_list2:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i.replace('-', ' ').upper()).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i.replace('-', ' ').upper()).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i.replace('-', ' ').upper()).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb3.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb3.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb3.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb3.update({i : ":("})

In [19]:
filtered_chembl3 = dict(filter(my_filtering_function, n_chemb3.items()))

In [20]:
filtered_list3 = list(filtered_chembl3.keys())

In [21]:
len(filtered_list3)

322

**Round 4**: change and add uppercases at the beginning

In [23]:
n_chemb4 = {}
for i in filtered_list3:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i.title()).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i.title()).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i.title()).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb4.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb4.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb4.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb4.update({i : ":("})

In [24]:
filtered_chembl4 = dict(filter(my_filtering_function, n_chemb4.items()))

## Transform results to work in R

In [27]:
def return_chembls(pair):
    unwanted_value = ':('
    key, value = pair
    if value == unwanted_value:
        return False  # filter pair out of the dictionary
    else:
        return True  # keep pair in the filtered dictionary
 

In [62]:
df_1 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb1.items())),orient = 'index')
df_2 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb2.items())),orient = 'index')
df_3 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb3.items())),orient = 'index')
df_4 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb4.items())),orient = 'index')

df_total = pd.concat([df_1,df_2,df_3,df_4])

In [63]:
df_total

,0
ribociclib,CHEMBL3545110
bortezomib,CHEMBL325041
linifanib,CHEMBL223360
dovitinib,CHEMBL522892
canertinib,CHEMBL31965
...,...
5-fluorouracil,CHEMBL185
se-methylselenocysteine,CHEMBL62382
skepinone-l,CHEMBL2152944
oncrasin-1,CHEMBL1775048


In [64]:
pd.DataFrame.to_csv(df_total,'output/chemblids_df_python.csv')


In [65]:
chembl_id = df_total[0].tolist()

In [72]:
len(np.unique(chembl_id))

1129

In [66]:
MOAs = {}

In [67]:
MOAs = {}
for i in chembl_id:
    mechanism = new_client.mechanism
    mols = mechanism.filter(molecule_chembl_id = i).only('mechanism_of_action')
    l = []
    if len(mols) > 0:
        for j in range(0,len(mols)):
            l.append((mols[j]['mechanism_of_action']))
            s = ', '.join(map(str, l))
            MOAs.update({i : s})
    else:
        MOAs.update({i : ":("})

In [68]:
len(MOAs)

1129

In [73]:
definition_chembl = {}
for i in chembl_id:
    molecule = new_client.molecule
    mols = molecule.filter(molecule_chembl_id = i).only('usan_stem_definition')
    if len(mols) > 0:
        definition_chembl.update({i : mols[0]["usan_stem_definition"]})
    else:
        definition_chembl.update({i : ":("})

In [74]:
df_moas_1 = pd.DataFrame.from_dict(MOAs,orient = 'index')
df_moas_2 = pd.DataFrame.from_dict(definition_chembl,orient = 'index')

In [75]:
moas_df  = pd.concat([df_moas_1, df_moas_2], axis=1)

In [76]:
moas_df.columns = ['MOAs','definitions']
moas_df

,MOAs,definitions
CHEMBL3545110,"Cyclin-dependent kinase 4 inhibitor, Cyclin-de...",cyclin dependent kinase inhibitors (formerly-c...
CHEMBL325041,26S proteosome inhibitor,proteozome inhibitors
CHEMBL223360,Macrophage colony stimulating factor receptor ...,angiogenesis inhibitors
CHEMBL522892,"Fibroblast growth factor receptor 3 inhibitor,...",tyrosine kinase inhibitors
CHEMBL31965,:(,tyrosine kinase inhibitors: epidermal growth f...
...,...,...
CHEMBL185,"Thymidylate synthase inhibitor, DNA inhibitor,...",uracil type antineoplastics; uracil derivative...
CHEMBL62382,:(,None
CHEMBL2152944,:(,None
CHEMBL1775048,:(,None


In [77]:
pd.DataFrame.to_csv(moas_df,'output/moas_df_python.csv')